In [1]:
# sample
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.1-70b-versatile"
)
response = llm.invoke("Hi how are you")
print(response.content)


I'm just a computer program, so I don't have feelings, but thanks for asking. How can I assist you today?


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/87773882722722502-senior-staff-software-engineer-onboarding-product-led-growth")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Senior Staff Software Engineer, Onboarding, Product Led Growth — Google CareersCareersSkip to main contentTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results2,714  jobs matchedSenior Security Engineer, Google Kubernetes EngineSeattle, WA, USA; Sunnyvale, CA, USAHealth and Home Strategy LeadMountain View, OR, USA; San Francisco, CA, USASoftware Engineering Manager II, Site Reliability Engineering, ShoppingDublin, IrelandSenior Staff Software Engineer, Onboarding, Product Led GrowthSeattle, WA, USA; San Francisco, CA, USA; +2 more; +1 moreProduct Manager, Meet Hardware, Google CloudNew York, NY, USA; Raleigh, NC, USA; +2 more; +1 moreStaff Network Engineer, Google Distributed CloudSunnyvale, CA, USAAI Sales SpecialistBerlin, Germany; Frankfurt, Germany;

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    #### Instruction
    the scraped text is from careers page of a website.
    your job is to extract necessary information relevant to job posting and return it in a JSON format container
    obtain the following keys : 'role','experience','skills' ans 'description'.
    ### Valid JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke({'page_data' : page_data})
print(res.content)

```json
{
  "role": "Senior Staff Software Engineer, Onboarding, Product Led Growth",
  "experience": {
    "minimum": {
      "bachelor_degree": true,
      "software_development": 8,
      "infrastructure_development": 7,
      "design_and_architecture": 5
    },
    "preferred": {
      "google_cloud_experience": true,
      "large_scale_distributed_systems": true
    }
  },
  "skills": {
    "programming_languages": ["TypeScript", "Java"],
    "technologies": ["Google Cloud", "AI/ML", "Generative AI"]
  },
  "description": "Provide technical leadership across the Google Cloud onboarding and product led growth organization. Drive expanded product usage and adoption through technological solutions driven by AI/ML. Develop technology strategy and phased plans for a platform enabling growth across Google Cloud. Partner with cross-functional leaders across Google Cloud to drive alignment and deliver customer value. Identify opportunities to leverage AI/ML, including Generative AI to imp

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Staff Software Engineer, Onboarding, Product Led Growth',
 'experience': {'minimum': {'bachelor_degree': True,
   'software_development': 8,
   'infrastructure_development': 7,
   'design_and_architecture': 5},
  'preferred': {'google_cloud_experience': True,
   'large_scale_distributed_systems': True}},
 'skills': {'programming_languages': ['TypeScript', 'Java'],
  'technologies': ['Google Cloud', 'AI/ML', 'Generative AI']},
 'description': 'Provide technical leadership across the Google Cloud onboarding and product led growth organization. Drive expanded product usage and adoption through technological solutions driven by AI/ML. Develop technology strategy and phased plans for a platform enabling growth across Google Cloud. Partner with cross-functional leaders across Google Cloud to drive alignment and deliver customer value. Identify opportunities to leverage AI/ML, including Generative AI to improve the cloud experiences.'}

In [9]:
import pandas as pd

df = pd.read_csv("app/resource/my_portfolio1.csv")
df

,Techstack,Links
0,"TypeScript, Java, Platform Extensions",https://example.com/typescript-java-platform-e...
1,"AI/ML, Generative AI, Cloud Computing",https://example.com/ai-ml-generativeai-cloudco...
2,"Distributed Systems, Networking, Data Storage",https://example.com/distributedsystems-network...
3,"Security, Natural Language Processing, UI Design",https://example.com/security-nlp-uidesign
4,"Mobile Development, AI/ML, Cloud Computing",https://example.com/mobiledevelopment-aiml-clo...
5,"JavaScript, Python, Data Analytics",https://example.com/javascript-python-dataanal...
6,"Blockchain, Smart Contracts, Cryptography",https://example.com/blockchain-smartcontracts-...
7,"Microservices, Docker, Kubernetes",https://example.com/microservices-docker-kuber...
8,"Big Data, Hadoop, Apache Spark",https://example.com/bigdata-hadoop-apachespark
9,"Virtualization, VMWare, OpenStack",https://example.com/virtualization-vmware-open...


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore1')
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links" : row["Links"]},
                       ids = [str(uuid.uuid4())]
                      )

In [13]:
links = collection.query(query_texts = ["Experinece in PHP", "Expertise in MVC"],
                         n_results = 2
                        ).get('metadatas',[])
links

[[{'links': 'https://example.com/security-nlp-uidesign'},
  {'links': 'https://example.com/ai-ml-generativeai-cloudcomputing'}],
 [{'links': 'https://example.com/backend-nodejs-rubyonrails'},
  {'links': 'https://example.com/frontend-react-angular'}]]

In [15]:
job = json_res
job['skills']

{'programming_languages': ['TypeScript', 'Java'],
 'technologies': ['Google Cloud', 'AI/ML', 'Generative AI']}

In [17]:
prompt_email = PromptTemplate.from_template(
             """   ### JOB DESCRIPTION:
            {job_description}

            ### INSTRUCTION:
            You are Raj, a senior business development executive at NexGen Solutions. NexGen Solutions is a leading AI & Software Consulting company focused on driving business innovation through advanced technology solutions. 
            With years of expertise, NexGen has helped numerous organizations streamline their processes, achieve scalability, and reduce operational costs. 
            Your task is to craft a cold email to the client based on the job description above, outlining how NexGen Solutions can meet their specific needs.
            Be sure to incorporate the most relevant case studies from the following links to highlight NexGen's portfolio: {link_list}
            Remember, you are Raj, Senior BDE at NexGen Solutions. 
            Avoid providing a preamble.
            ### EMAIL (NO PREAMBLE) """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking AI-Driven Growth for Google Cloud Onboarding and Product Led Growth

Dear Hiring Manager,

I came across the Senior Staff Software Engineer, Onboarding, Product Led Growth role at Google Cloud and was impressed by the organization's commitment to driving technological innovation through AI/ML. As a seasoned business development executive at NexGen Solutions, I'd like to highlight how our expertise can support your goals.

Our team has extensive experience in developing scalable, AI-driven solutions that enhance customer experiences. I'd like to draw your attention to our recent success story in AI/ML and Generative AI for cloud computing, where we leveraged cutting-edge technologies to create a cloud-based platform that improved operational efficiency by 30% (https://example.com/ai-ml-generativeai-cloudcomputing). This project showcases our ability to design and implement AI-driven solutions that drive business growth.

Furthermore, our expertise in backend developme